In [1]:
from down_data import Downloader
import pandas as pd
import json

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.options.display.max_colwidth = 100
%matplotlib inline

import plotly
import plotly.graph_objs as go

import numpy as np

In [2]:
down = Downloader()

In [16]:
def plot_dfs_3D(serX, serY):
    X = pd.DataFrame(serX)
    X = X.transpose()
    X = X.append([X]*len(X.columns), ignore_index=True)[0:-1]
    
    Y = pd.DataFrame(serY)
    Y = Y.transpose()
    Y = Y.append([Y]*len(Y.columns), ignore_index=True)[0:-1]
    Y = Y.transpose()
    
    Z = X.values * Y.values
    
    df_Z = pd.DataFrame(Z)
    data = [go.Surface(z=df_Z.values.tolist(), colorscale='Viridis')]

    layout = go.Layout(
        width=900,
        height=800,
        autosize=False,
        title='BTC_USD to BTC_USD',
        scene=dict(
            xaxis=dict(
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            yaxis=dict(
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            zaxis=dict(
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            aspectratio = dict( x=1, y=1, z=0.7 ),
            aspectmode = 'manual'
        )
    )
    
    fig = dict(data=data, layout=layout)
    url = plotly.offline.plot(fig)

In [15]:
def plot_series_2D(ser1, ser2):
    trace = go.Scatter(
        x = ser1.tolist(),
        y = ser2.tolist(),
        mode = 'lines+markers',
        name = 'plot_series_2D'
    )
    
    data = [trace]
    plotly.offline.plot(data)

In [11]:
price_BTC_USD = down.get_pricemulti_json(['BTC'], ['USD'], e='Poloniex')

histo_json_XMR_USD = down.get_histo_json('XMR', 'USD', limit=24*1, t_horiz='hour') #max limit = 24*83
#histo_json_XMR_BTC = down.get_histo_json('XMR', 'BTC', limit=24*1, t_horiz='hour') #max limit = 24*83

tradeHistory_XMR_USD = down.get_tradeHistory_json(['BTC', 'XMR'], t_from=histo_json_XMR_USD['TimeFrom'], t_to=histo_json_XMR_USD['TimeTo'])

get_pricemulti request: https://min-api.cryptocompare.com/data/pricemulti?&fsyms=BTC&tsyms=USD&e=Poloniex
get_histohour request: https://min-api.cryptocompare.com/data/histohour?&fsym=XMR&tsym=USD&limit=24&aggregate=1&e=Poloniex
get_returnTradeHistory request: https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_XMR&start=1512658800&end=1512745200


In [5]:
df_XMR_USD = pd.DataFrame(histo_json_XMR_USD['Data'])
df_XMR_USD = df_XMR_USD[['time', 'close', 'high', 'low', 'open']]
df_XMR_USD['time'] = pd.to_datetime(df_XMR_USD['time'], unit='s')

df_XMR_USD_diff = df_XMR_USD[['close', 'high', 'low', 'open']].diff()[1:-1]
df_XMR_USD_diff['time'] = df_XMR_USD['time'][1:-1]
df_XMR_USD_diff = df_XMR_USD_diff.reset_index(drop=True)

In [6]:
# df_XMR_BTC = pd.DataFrame(histo_json_XMR_BTC['Data'])
# df_XMR_BTC = df_XMR_BTC[['time', 'close', 'high', 'low', 'open']]
# df_XMR_BTC['time'] = pd.to_datetime(df_XMR_BTC['time'], unit='s')

# df_XMR_BTC_diff = df_XMR_BTC[['close', 'high', 'low', 'open']].diff()[1:-1]
# df_XMR_BTC_diff['time'] = df_XMR_BTC['time'][1:-1]
# df_XMR_BTC_diff = df_XMR_BTC_diff.reset_index(drop=True)

NameError: name 'histo_json_XMR_BTC' is not defined

In [ ]:
plot_dfs_3D(df_XMR_USD, df_XMR_BTC)

In [13]:
df_trade = pd.DataFrame(tradeHistory_XMR_USD)

In [20]:
df_trade.head()

,amount,date,globalTradeID,rate,total,tradeID,type
0,0.09343977,2017-12-08 14:59:47,282068316,0.01603709,0.00149850,14553314,sell
1,0.08920773,2017-12-08 14:59:16,282067921,0.01603709,0.00143063,14553313,sell
2,0.50000000,2017-12-08 14:59:05,282067786,0.01604460,0.00802230,14553312,buy
3,1.13693319,2017-12-08 14:58:49,282067578,0.01603709,0.01823309,14553311,sell
4,0.05901054,2017-12-08 14:58:49,282067577,0.01604360,0.00094674,14553310,sell


In [21]:
plot_series_2D(df_XMR_USD_diff.close, df_trade.total)